In [26]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [15]:
data = pd.read_csv('../data/processed/processed_data.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289198 entries, 0 to 289197
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      289198 non-null  int64 
 1   text            289198 non-null  object
 2   y               289198 non-null  int64 
 3   processed_text  288376 non-null  object
dtypes: int64(2), object(2)
memory usage: 8.8+ MB


In [16]:
data.drop(columns=['Unnamed: 0', 'text'], inplace=True)
data.dropna(inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 288376 entries, 0 to 289197
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   y               288376 non-null  int64 
 1   processed_text  288376 non-null  object
dtypes: int64(1), object(1)
memory usage: 6.6+ MB


In [17]:
tfidf = TfidfVectorizer(
    max_features=5000,      # Топ-5000 самых частых слов
    min_df=5,               # Игнорируем слова, встречающиеся реже 5 раз
    max_df=0.9,             # Игнорируем слова, которые есть в 90% док-тов
    ngram_range=(1, 2)      # Униграммы и Биграммы
)

# X - это готовая матрица признаков для модели
X = tfidf.fit_transform(data['processed_text']) 
y = data['y']

# ВАЖНО для MLOps: Сохраните векторизатор!
joblib.dump(tfidf, '../models/vectorizers/tfidf_vectorizer.pkl')

['../models/vectorizers/tfidf_vectorizer.pkl']

In [ ]:
# 1. Загружаем предобученную модель (она скачается один раз автоматически)
model_name = 'cointegrated/rubert-tiny2'
emb_model = SentenceTransformer(model_name, device='cpu')

print("Модель загружена")

Модель загружена
Модель сохранена в ../models/vectorizers/rubert-tiny2


In [ ]:
embeddings = emb_model.encode(data['processed_text'].tolist(), show_progress_bar=True)

data['embeddings'] = list(embeddings)

data.to_pickle('../data/processed/data_with_embeddings.pkl')
print("Датасет с эмбеддингами сохранен!")

print(f"Размерность матрицы: {embeddings.shape}") 

Batches: 100%|██████████| 9012/9012 [11:26<00:00, 13.14it/s] 


Размерность матрицы: (288376, 312)


In [32]:
# Разделение на Train/Test
X_train, X_test, y_train, y_test = train_test_split(embeddings, data['y'], test_size=0.2, random_state=42)

# Обучение модели
# max_iter=1000 нужно, чтобы модель успела сойтись
clf = LogisticRegression(random_state=42, max_iter=10000)
clf.fit(X_train, y_train)

# Предсказание
y_pred = clf.predict(X_test)

# Оценка
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98     28058
           1       0.98      0.99      0.98     29618

    accuracy                           0.98     57676
   macro avg       0.98      0.98      0.98     57676
weighted avg       0.98      0.98      0.98     57676



In [33]:
# Сохраняем классификатор (он будет весить копейки, килобайты)
joblib.dump(clf, '../models/logreg_classifier.pkl')

print("Классификатор сохранен")

Классификатор сохранен
